In [1]:
%%capture
!pip install --quiet "torchmetrics >=1.0,<1.5" "seaborn" "numpy <3.0" "torchvision" "tensorboard" "torch >=1.8.1,<2.5" "pytorch-lightning >=2.0,<2.5" "matplotlib"

In [2]:
import os
import urllib.request
from copy import deepcopy
from urllib.error import HTTPError

import matplotlib
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
import pytorch_lightning as pl
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from torchvision import transforms
from torchvision.datasets import STL10
from tqdm.notebook import tqdm

plt.set_cmap("cividis")
%matplotlib inline
matplotlib_inline.backend_inline.set_matplotlib_formats("svg", "pdf")  # For export
matplotlib.rcParams["lines.linewidth"] = 2.0
sns.set()

# Import tensorboard
%load_ext tensorboard

<Figure size 640x480 with 0 Axes>

In [3]:
## Data Preparation

# Path configurations
DATASET_PATH = "data/"
NUM_WORKERS = os.cpu_count()

# Data augmentations for SimCLR
contrast_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(size=96),
    transforms.RandomApply([
        transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1)
    ], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.GaussianBlur(kernel_size=9),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Datasets for contrastive learning
unlabeled_data = STL10(
    root=DATASET_PATH,
    split="unlabeled",
    download=True,
    transform=contrast_transforms
)
train_data_contrast = STL10(
    root=DATASET_PATH,
    split="train",
    download=True,
    transform=contrast_transforms
)


100%|██████████| 2640397119/2640397119 [02:45<00:00, 15989262.28it/s]


Extracting data/stl10_binary.tar.gz to data/
Files already downloaded and verified


In [8]:
## SimCLR model definition

class SimCLR(pl.LightningModule):
    def __init__(self, hidden_dim=128, lr=1e-3, temperature=0.07, weight_decay=1e-4, max_epochs=100):
        super().__init__()
        self.save_hyperparameters()

        # INTERNAL NOTES
        # 1. In the future, try with other backbones (could be Resnet50, could be some EfficientNet)
        # 2. Since SimCLR learns representations directly from the data, I'm not using the pre-trained weights by now
        # to avoid the bias learned from ImageNet-like datasets. However, this should also be tested.
        # Tip: for larger datasets, pretrained=False should work better (because of what I've just exposed)
        # while for small datasets, the pretrained weights might provide a performance boost
        # Define ResNet backbone
        self.convnet = torchvision.models.resnet18(pretrained=False)
        self.convnet.fc = nn.Sequential(
            nn.Linear(self.convnet.fc.in_features, 4 * hidden_dim),
            nn.ReLU(inplace=True),
            nn.Linear(4 * hidden_dim, hidden_dim)
        )

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=self.hparams.lr, weight_decay=self.hparams.weight_decay)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=self.hparams.max_epochs, eta_min=self.hparams.lr / 50)
        return [optimizer], [scheduler]

    def info_nce_loss(self, batch):
        imgs, _ = batch
        # imgs = torch.cat(imgs, dim=0)
        feats = self.convnet(imgs)
        cos_sim = F.cosine_similarity(feats[:, None, :], feats[None, :, :], dim=-1)
        self_mask = torch.eye(cos_sim.shape[0], device=cos_sim.device, dtype=torch.bool)
        cos_sim.masked_fill_(self_mask, -9e15)
        pos_mask = self_mask.roll(shifts=cos_sim.shape[0] // 2, dims=0)
        cos_sim = cos_sim / self.hparams.temperature
        nll = -cos_sim[pos_mask] + torch.logsumexp(cos_sim, dim=-1)
        return nll.mean()

    def training_step(self, batch, batch_idx):
        loss = self.info_nce_loss(batch)
        self.log("train_loss", loss)
        return loss


In [ ]:
## Train SimCLR Model

def train_simclr(batch_size=256, max_epochs=100, **kwargs):
    trainer = pl.Trainer(
        max_epochs=max_epochs,
        accelerator="gpu",
        devices=1,
        callbacks=[
            ModelCheckpoint(save_weights_only=True, monitor="train_loss", mode="min"),
            LearningRateMonitor(logging_interval="epoch"),
        ]
    )
    train_loader = torch.utils.data.DataLoader(unlabeled_data, batch_size=batch_size, shuffle=True, num_workers=NUM_WORKERS)
    model = SimCLR(max_epochs=max_epochs, **kwargs)
    trainer.fit(model, train_loader)
    return model

# Train the SimCLR model
simclr_model = train_simclr(hidden_dim=128, lr=1e-3, temperature=0.07, weight_decay=1e-4, max_epochs=100)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type   | Params | Mode 
-------------------------------------------
0 | convnet | ResNet | 11.5 M | train
-------------------------------------------
11.5 M    Trainable params
0         Non-trainable params
11.5 M    Total params
46.019    Total estimated model params size (MB)
71        Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

In [ ]:
## Save the model

torch.save(simclr_model.state_dict(), "simclr_model.pth")

In [ ]:
## Define MLP Classifier

class MLPClassifier(pl.LightningModule):
    def __init__(self, feature_dim, num_classes, hidden_dim=256, lr=1e-3, weight_decay=1e-4, max_epochs=100):
        super().__init__()
        self.save_hyperparameters()

        # Define the MLP architecture
        self.model = nn.Sequential(
            nn.Linear(feature_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, num_classes)
        )

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=self.hparams.lr, weight_decay=self.hparams.weight_decay)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=self.hparams.max_epochs, eta_min=self.hparams.lr / 50)
        return [optimizer], [scheduler]

    def _calculate_loss(self, batch, mode="train"):
        feats, labels = batch
        preds = self.model(feats)
        loss = F.cross_entropy(preds, labels)
        acc = (preds.argmax(dim=-1) == labels).float().mean()

        # Log loss and accuracy
        self.log(f"{mode}_loss", loss, prog_bar=True)
        self.log(f"{mode}_acc", acc, prog_bar=True)
        return loss

    def training_step(self, batch, batch_idx):
        return self._calculate_loss(batch, mode="train")

    def validation_step(self, batch, batch_idx):
        self._calculate_loss(batch, mode="val")

    def test_step(self, batch, batch_idx):
        self._calculate_loss(batch, mode="test")

In [ ]:
## Train the MLP Classifier

def train_mlp(batch_size, train_feats_data, test_feats_data, max_epochs=100, **kwargs):
    trainer = pl.Trainer(
        max_epochs=max_epochs,
        accelerator="gpu",
        devices=1,
        callbacks=[
            pl.callbacks.ModelCheckpoint(save_weights_only=True, monitor="val_acc", mode="max"),
            pl.callbacks.LearningRateMonitor("epoch"),
        ],
    )

    # Data loaders
    train_loader = torch.utils.data.DataLoader(
        train_feats_data, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=4
    )
    test_loader = torch.utils.data.DataLoader(
        test_feats_data, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=4
    )

    # Train the MLP
    model = MLPClassifier(max_epochs=max_epochs, **kwargs)
    trainer.fit(model, train_loader, test_loader)

    # Test the model
    test_result = trainer.test(model, test_loader, verbose=False)
    print(f"Test accuracy: {test_result[0]['test_acc'] * 100:.2f}%")

    return model


In [6]:
!pip install captum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.3 MB/s eta 0:00:00


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from captum.attr import IntegratedGradients

def run_explainability():
    print("Loading model...")
    model = torch.load(f='simclr_model.pth', weights_only=False)
    # model.eval()
    # Select the target class for which you want to explain predictions
    dataset = train_data_contrast
    input_tensor = torch.from_numpy(dataset[0].data)
    # target_class = torch.argmax(model(input_tensor)).item()
    print("Got an example predicted with class 0")
    ig = IntegratedGradients(model)
    attributions, delta = ig.attribute(
        input_tensor, target=0, return_convergence_delta=True
    )
    attributions_np = attributions[0].cpu().detach().numpy()

    attributions_np = (attributions_np - attributions_np.min()) / (attributions_np.max() - attributions_np.min())

    print("Preparing image with attributions")
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(input_tensor[0].cpu().permute(1, 2, 0))
    ax[0].set_title('Original Image')
    ax[1].imshow(attributions_np.transpose(1, 2, 0))
    ax[1].set_title('Attributions')
    plt.show()


In [ ]:
run_explainability()